Note that this code requires a reasonably good graphics card to run. 

In [1]:
import pmcx
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
import pandas as pd

In [2]:
DX = 0.06  # mm
NX = 1000

In [ ]:
# Scattering properties:
def mus_prime(wavelengths):
    return 22 * (wavelengths / 500) ** (-0.66) / 10  # /mm


g = 0.9


def mus_bg(wavelengths):
    return mus_prime(wavelengths) / (1 - g)


# Blood spectrum and melanin spectrum:
specs = np.load("HbSpec.npz")  # in /cm /M


def hb(x):
    return (
        np.interp(x, specs["wavelengths"], specs["hb"]) * 0.1 * 2.303 * 150 / 64500
    )  # convert to /mm


def hbo2(x):
    return (
        np.interp(x, specs["wavelengths"], specs["hbo2"]) * 0.1 * 2.303 * 150 / 64500
    )  # convert to /mm


def mua_blood(wavelengths, oxy=0.9, bvf=0.03):
    return bvf * (hb(wavelengths) * (1 - oxy) + hbo2(wavelengths) * oxy) / 1000  # /mm


def mua_melanin(x, mvf):
    return 519 * (x / 500) ** (-3.5) * mvf / 10  # /mm

In [4]:
def run_acuity_sim(
    mua_skin,
    mus_skin,
    g_skin,
    n_skin,
    mua_bg,
    mus_bg,
    g_bg,
    n_bg,
    mua_blood,
    n_ext=1,
    nz=100,
    nx=100,
    dx=0.5,
    dt=5e-9,
    nphotons=1e6,
):
    ## ACUITY MODELLING ##
    divergence_angle = (
        8.66  # full beam divergence angle measured at Full Width at Half Maximum (FWHM)
    )
    full_width_at_half_maximum = 2.0 * np.tan(
        0.5 * np.deg2rad(divergence_angle)
    )  # FWHM of beam divergence
    # standard deviation of gaussian with FWHM
    sigma = full_width_at_half_maximum / (2.0 * np.sqrt(2.0 * np.log(2.0)))

    width = 30 / dx

    # Angle of incidence of the light:
    srcdir = [0.0, np.sin(np.deg2rad(22.4)), np.cos(np.deg2rad(22.4))]
    srcpos = [
        nx / 2 - width / 2,
        nx / 2 - (43.2 + 2.8) / dx * np.tan(np.deg2rad(22.4)),
        nx / 2 - 43.2 / dx,
    ]  # Center of image is the imaging plane
    print(srcpos)
    # volume definition:
    vol = np.ones([nx, nx, nz], dtype="uint8")
    vol[:, :, : nz // 2] = 5  # Dermis
    vol[:, :, : nz // 2 - int(1 / dx)] = 3  # Background
    vol[:, :, nz // 2] = 2

    # Add a blood vessel, which is where we will measure the light fluence
    z = np.arange(nz) * dx
    x = np.arange(nx) * dx

    z0 = z[nz // 2] + 5
    x0 = x[nx // 2]
    r = 1.5
    selection = (z[None, None] - z0) ** 2 + (x[:, None, None] - x0) ** 2 < r**2
    vol[np.broadcast_to(selection, vol.shape)] = 4

    mua = np.array([1e-10, mua_bg, mua_skin, 1e-10, mua_blood, 0.002 * mua_bg / 0.025])
    mus = np.array([1e-10, mus_bg, mus_skin, 1e-10, mus_bg, mus_bg])
    g = np.array([1, g_bg, g_skin, 1, g_bg, g_bg])
    n = np.array([n_ext, n_bg, n_skin, n_ext, n_bg, n_bg])

    cfg = {
        "nphoton": nphotons,
        "vol": vol,
        "tstart": 0,
        "tend": dt,
        "tstep": dt,
        "srctype": "slit",
        "srcpos": srcpos,  # [nx/2-width/2, nx/2,-10],
        "srcdir": srcdir,
        "srcparam1": [width, 0.0, 0.0, 0.0],
        "srcparam2": [sigma, sigma, 0.0, 0.0],
        "prop": [[a, b, c, d] for a, b, c, d in zip(mua, mus, g, n)],
        "unitinmm": dx,
        "issrcfrom0": True,
        "isnormalize": True,
    }
    res = pmcx.run(cfg)
    res["flux"] *= dt
    fluence = np.sum(res["flux"], axis=-1) * dx**2
    return res, fluence, fluence * mua[vol], vol, mua[vol]


def run_skin_simulation(wavelength, mvf):
    mus_bg = mus_prime(wavelength) / (1 - g)
    nx = NX
    dx = DX
    oxy = 0.7
    bvf = 0.025

    mua_bg = mua_blood(wavelength, oxy=oxy, bvf=bvf)
    mua_skin = mua_melanin(wavelength, mvf) + mua_bg

    res, fluence, p0, vol, mua = run_acuity_sim(
        mua_skin,
        mus_bg,
        g,
        1.37,
        mua_bg,
        mus_bg,
        g,
        1.37,
        mua_blood(wavelength, 1, 1),
        nphotons=1e7,
        nx=nx,
        dx=dx,
        nz=nx,
    )
    return res, fluence, p0, vol, mua

In [ ]:
# for mvf, wl
res_a, fluence_a, p0, vol, mua = run_skin_simulation(700, 0.4)

[250.0, 184.0028029103148, -220.0000000000001]
nphoton: 1e+07
tstart: 0
tstep: 5e-09
tend: 5e-09
issrcfrom0: 1
unitinmm: 0.06


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(8, 2.5))
im = axes[0].imshow(
    np.log(np.mean(fluence_a, axis=0).T),
    extent=(-DX * NX / 20, DX * NX / 20, -DX * NX / 20, DX * NX / 20),
)
axes[0].set_title("side (z-y)")
axes[1].imshow(
    np.log(np.mean(fluence_a[:, :, NX // 2 : NX // 2 + 6], axis=2).T),
    extent=(-DX * NX / 20, DX * NX / 20, -DX * NX / 20, DX * NX / 20),
    clim=im.get_clim(),
)
axes[1].set_title("top down (x-y)")
axes[2].imshow(
    np.log(np.mean(fluence_a, axis=1).T),
    extent=(-DX * NX / 20, DX * NX / 20, -DX * NX / 20, DX * NX / 20),
    clim=im.get_clim(),
)
axes[2].set_title("side (z-x)")
fig.suptitle("Fluence")
plt.colorbar(im, ax=axes, label="Log(Light Fluence)")
# plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(6, 2.5))
im = axes[2].imshow(
    np.log(np.mean(p0, axis=1).T),
    extent=(-DX * NX / 20, DX * NX / 20, -DX * NX / 20, DX * NX / 20),
)
axes[2].set_title("side (z-x)")
axes[0].imshow(
    np.log(np.mean(p0, axis=0).T),
    extent=(-DX * NX / 20, DX * NX / 20, -DX * NX / 20, DX * NX / 20),
    clim=im.get_clim(),
)
axes[0].set_title("side (z-y)")
axes[1].imshow(
    np.log(np.mean(p0[:, :, NX // 2 : NX // 2 + 6], axis=2)),
    extent=(-DX * NX / 20, DX * NX / 20, -DX * NX / 20, DX * NX / 20),
    clim=im.get_clim(),
)
axes[1].set_title("top down (x-y)")
fig.suptitle("Absorption")
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(8, 2.5))
im = axes[2].imshow(
    np.mean(vol, axis=1).T,
    extent=(-DX * NX / 20, DX * NX / 20, -DX * NX / 20, DX * NX / 20),
    interpolation="none",
)
axes[2].set_title("side (z-x)")
axes[0].imshow(
    np.mean(vol, axis=0).T,
    extent=(-DX * NX / 20, DX * NX / 20, -DX * NX / 20, DX * NX / 20),
    clim=im.get_clim(),
    interpolation="none",
)
axes[0].set_title("side (z-y)")
axes[1].imshow(
    np.mean(vol[:, :, NX // 2 : NX // 2 + 6], axis=2),
    extent=(-DX * NX / 20, DX * NX / 20, -DX * NX / 20, DX * NX / 20),
    clim=im.get_clim(),
    interpolation="none",
)
axes[1].set_title("top down (x-y)")
fig.suptitle("Volume")
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(8, 2.5))
im = axes[2].imshow(
    np.log(np.mean(mua, axis=1).T),
    extent=(-DX * NX / 20, DX * NX / 20, -DX * NX / 20, DX * NX / 20),
)
axes[2].set_title("side (z-x)")
axes[0].imshow(
    np.log(np.mean(mua, axis=0).T),
    extent=(-DX * NX / 20, DX * NX / 20, -DX * NX / 20, DX * NX / 20),
    clim=im.get_clim(),
)
axes[0].set_title("side (z-y)")
axes[1].imshow(
    np.log(np.mean(mua[:, :, NX // 2 : NX // 2 + 6], axis=2)),
    extent=(-DX * NX / 20, DX * NX / 20, -DX * NX / 20, DX * NX / 20),
    clim=im.get_clim(),
)
axes[1].set_title("top down (x-y)")
fig.suptitle("mua")
plt.colorbar(im, ax=axes, label=r"Log($\mu_a$)")
plt.show()

# Make a calibration curve for the blood vessel's fluence

In [ ]:
mvfs = np.logspace(np.log10(0.02), np.log10(0.4), 6)
wavelengths = np.linspace(700, 900, 5)
# TODO: consider doing the diffuse reflectance simulations too, to get ITA a' la previous paper

In [ ]:
params_results = []
for mvf, wl in product(mvfs, wavelengths):
    print(mvf, wl)
    res_a, fluence_a, p0, vol, mua = run_skin_simulation(wl, mvf)
    vessel_fluence = np.mean(fluence_a[:, NX // 2][vol[:, NX // 2] == 4])
    result = {}
    result["MVF"] = mvf
    result["WL"] = wl
    result["Fluence"] = vessel_fluence
    params_results.append(result)

In [ ]:
df = pd.DataFrame(params_results)
df.to_csv("cali_curve.csv")
# Previously, the code in this code was run on a separate PC, hence this strange extra line:
df.to_csv("../cali_curve.csv")